In [1]:
import requests
import pandas as pd
import json
from io import StringIO
from datetime import datetime, timedelta

### Investing.com 에서 주가 데이터 불러오기

curr_id : 인베스팅닷컴에서 부여한 고유의 ID
    
    
따라서 curr_id 를 가져오는 단계가 반드시 필요하다.

------------

curr_id는 어디서 가져올까?


검색창에서 종목 코드를 넣으면 curr_id를 찾을 수 있음


그 이후에 데이터를 요청하면 됨

### 사이트 특징

1) curr_id : 사이트가 고유로 부여한 아이디.... 검색창을 통해서 구한다.
    
    
2) header 정보 넣기..... 하나하나 해보면 구할 수 있다.




In [2]:
import requests
import pandas as pd
import json
from io import StringIO
from datetime import datetime, timedelta


In [3]:
url = 'https://www.investing.com/instruments/HistoricalDataAjax'

In [4]:
data = {
    'curr_id': '6408',
    'smlID': '1159963',
    'header': 'AAPL Historical Data',
    'st_date': '04/27/2016',
    'end_date': '08/04/2018',
    'interval_sec': 'Daily',
    'sort_col': 'date',
    'sort_ord': 'DESC',
    'action': 'historical_data'
}

In [5]:
headers = {
    'User-Agent':'Mozilla',
    'X-Requested-With':'XMLHttpRequest',

}

In [6]:
res = requests.post(url, headers = headers, data = data)

In [7]:
pd.read_html(res.text)

[             Date   Price    Open    High     Low     Vol. Change %
 0    Aug 03, 2018  207.99  207.03  208.74  205.48   33.45M    0.29%
 1    Aug 02, 2018  207.39  200.58  208.38  200.35   62.40M    2.92%
 2    Aug 01, 2018  201.50  199.13  201.76  197.31   67.94M    5.89%
 3    Jul 31, 2018  190.29  190.30  192.14  189.34   39.37M    0.20%
 4    Jul 30, 2018  189.91  191.90  192.20  189.07   21.03M   -0.56%
 5    Jul 27, 2018  190.98  194.99  195.19  190.10   24.02M   -1.66%
 6    Jul 26, 2018  194.21  194.61  195.96  193.61   19.08M   -0.31%
 7    Jul 25, 2018  194.82  193.06  194.85  192.43   16.83M    0.94%
 8    Jul 24, 2018  193.00  192.45  193.66  192.05   18.70M    0.73%
 9    Jul 23, 2018  191.61  190.68  191.96  189.56   15.99M    0.09%
 10   Jul 20, 2018  191.44  191.78  192.43  190.17   20.71M   -0.23%
 11   Jul 19, 2018  191.88  189.69  192.55  189.69   20.29M    0.78%
 12   Jul 18, 2018  190.40  191.78  191.80  189.93   16.39M   -0.55%
 13   Jul 17, 2018  191.45  189.75

#### 회사 ID 찾기

In [8]:
def find_curr_id(code) :
    url = 'https://www.investing.com/search/service/search'
    headers = {
            'User-Agent':'Mozilla',
            'X-Requested-With':'XMLHttpRequest',
        }
    
    datas = {
        'search_text': code,
        'term': code,
        'country_id': '0',
        'tab_id': 'All'
        }
    
    res = requests.post(url, data=datas, headers=headers)
    
    code_dict = json.loads(res.text)
    
   
    return str(list(filter(lambda x : x['symbol'] == code, code_dict['All']))[0]['pair_ID'])

In [9]:
curr_id = find_curr_id('AAPL')
curr_id

'6408'

### 주가 불러오기

#### 스타트 날짜 설정(양식은 dd/mm/yyyy로 통일할 것)

In [10]:
start_date = "01/01/2017" # 날짜 양식은 dd/mm/yyyy 로 통일할 것

#### 금일 날짜를 양식에 맞게 변경

In [11]:
today = datetime.strftime(datetime.utcnow() + timedelta(hours = 9), "%m/%d/%Y")
today

'04/13/2019'

In [12]:
url = 'https://www.investing.com/instruments/HistoricalDataAjax'

In [13]:
data = {    
            'curr_id':curr_id,
            'st_date': start_date,
            'end_date': today,
            'interval_sec':'Daily',
            'sort_col':'date',
            'sort_ord':'ASC',
            'action':'historical_data'
        }


In [14]:
headers = {
    'User-Agent':'Mozilla',
    'X-Requested-With':'XMLHttpRequest',
    }


In [15]:
r = requests.post(url, data, headers=headers)

In [16]:
pd.read_html(r.text)[0]

,Date,Price,Open,High,Low,Vol.,Change %
0,"Apr 12, 2019",197.31,199.22,200.11,196.31,13.82M,-0.91%
1,"Apr 11, 2019",199.12,200.70,200.94,198.49,392.81K,-0.75%
2,"Apr 10, 2019",200.62,198.68,200.74,198.18,21.70M,0.56%
3,"Apr 09, 2019",199.50,200.32,202.85,199.23,35.77M,-0.30%
4,"Apr 08, 2019",200.10,196.42,200.23,196.34,25.88M,1.57%
5,"Apr 05, 2019",197.00,196.45,197.10,195.93,18.53M,0.67%
6,"Apr 04, 2019",195.69,194.79,196.37,193.14,19.11M,0.17%
7,"Apr 03, 2019",195.35,193.25,196.50,193.15,23.27M,0.69%
8,"Apr 02, 2019",194.02,191.09,194.46,191.05,22.77M,1.45%
9,"Apr 01, 2019",191.24,191.64,191.68,188.38,27.86M,0.68%


### 최종함수 (코드를 받아서 일간 데이터로 변경)

In [17]:
def get_historical_data(code, start_date) :

    # 기업 인베스팅닷컴 고유 아이디를 구하기
    url = 'https://www.investing.com/search/service/search'
    headers = {
            'User-Agent':'Mozilla',
            'X-Requested-With':'XMLHttpRequest',
        }
    
    datas = {
        'search_text': code,
        'term': code,
        'country_id': '0',
        'tab_id': 'All'
        }
    res = requests.post(url, data=datas, headers=headers)
    
    
    # 해당 데이에서 curr_id 구하기
    code_dict = json.loads(res.text)
    
   
    curr_id = str(list(filter(lambda x : x['symbol'] == code, code_dict['All']))[0]['pair_ID'])
    
    today = datetime.strftime(datetime.utcnow() + timedelta(hours = 9), "%m/%d/%Y")
    
    
    # 주가가져오기

    url_data = 'https://www.investing.com/instruments/HistoricalDataAjax'
    
    datas = {    
                'curr_id': curr_id,   # 앞에서 구한 기업 고유 아이디
                'st_date': start_date,  # 함수에서 입력받은 날짜
                'end_date': today,   # 오늘날짜(변경가능)
                'interval_sec':'Daily',
                'sort_col':'date',
                'sort_ord':'ASC',
                'action':'historical_data'
            }

    headers = {
            'User-Agent':'Mozilla',
            'X-Requested-With':'XMLHttpRequest',
            }

    res_data = requests.post(url_data, data=datas, headers=headers)
    
    
    return pd.read_html(res_data.text)[0]    # 데이터 데이블 받기

In [18]:
start_date = '01/01/2015'
code = '7203'

price_table = get_historical_data(code, start_date) 



In [19]:
price_table['Date'] = pd.to_datetime(price_table['Date'])

In [20]:
price_table[::-1]

,Date,Price,Open,High,Low,Vol.,Change %
1048,2015-01-05,7507.0,7565.0,7575.0,7416.0,9.52M,-0.67%
1047,2015-01-06,7300.0,7322.0,7391.0,7300.0,12.39M,-2.76%
1046,2015-01-07,7407.0,7256.0,7485.0,7255.0,11.47M,1.47%
1045,2015-01-08,7554.0,7500.0,7556.0,7495.0,10.05M,1.98%
1044,2015-01-09,7609.0,7630.0,7666.0,7561.0,10.43M,0.73%
1043,2015-01-13,7519.0,7440.0,7526.0,7368.0,11.11M,-1.18%
1042,2015-01-14,7396.0,7428.0,7479.0,7390.0,10.03M,-1.64%
1041,2015-01-15,7526.0,7437.0,7548.0,7435.0,8.93M,1.76%
1040,2015-01-16,7514.0,7403.0,7534.0,7393.0,12.78M,-0.16%
1039,2015-01-19,7465.0,7534.0,7547.0,7450.0,9.93M,-0.65%


In [21]:
price_table[price_table['Price']<100000]

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-04-12,6789.0,6785.0,6797.0,6759.0,3.45M,0.71%
1,2019-04-11,6741.0,6793.0,6803.0,6741.0,3.81M,-0.79%
2,2019-04-10,6795.0,6791.0,6810.0,6767.0,2.92M,-0.53%
3,2019-04-09,6831.0,6800.0,6835.0,6784.0,3.16M,0.63%
4,2019-04-08,6788.0,6850.0,6851.0,6763.0,3.75M,-0.76%
5,2019-04-05,6840.0,6779.0,6840.0,6768.0,4.82M,0.97%
6,2019-04-04,6774.0,6750.0,6788.0,6739.0,3.76M,0.68%
7,2019-04-03,6728.0,6706.0,6742.0,6666.0,4.43M,0.58%
8,2019-04-02,6689.0,6699.0,6714.0,6660.0,4.51M,0.65%
9,2019-04-01,6646.0,6571.0,6685.0,6557.0,6.26M,2.45%


In [22]:
price_table['Vol.'].describe()

count      1049
unique      659
top       5.25M
freq          7
Name: Vol., dtype: object

In [23]:
price_table[price_table['Vol.'].str.contains('-') == False]

,Date,Price,Open,High,Low,Vol.,Change %
0,2019-04-12,6789.0,6785.0,6797.0,6759.0,3.45M,0.71%
1,2019-04-11,6741.0,6793.0,6803.0,6741.0,3.81M,-0.79%
2,2019-04-10,6795.0,6791.0,6810.0,6767.0,2.92M,-0.53%
3,2019-04-09,6831.0,6800.0,6835.0,6784.0,3.16M,0.63%
4,2019-04-08,6788.0,6850.0,6851.0,6763.0,3.75M,-0.76%
5,2019-04-05,6840.0,6779.0,6840.0,6768.0,4.82M,0.97%
6,2019-04-04,6774.0,6750.0,6788.0,6739.0,3.76M,0.68%
7,2019-04-03,6728.0,6706.0,6742.0,6666.0,4.43M,0.58%
8,2019-04-02,6689.0,6699.0,6714.0,6660.0,4.51M,0.65%
9,2019-04-01,6646.0,6571.0,6685.0,6557.0,6.26M,2.45%


In [24]:
get_historical_data('TSLA', '01/04/2018')

,Date,Price,Open,High,Low,Vol.,Change %
0,"Apr 12, 2019",267.92,269.95,271.86,267.66,3.29M,-0.19%
1,"Apr 11, 2019",268.42,268.00,270.38,265.83,9.84M,-2.77%
2,"Apr 10, 2019",276.06,276.74,278.38,272.89,7.06M,1.38%
3,"Apr 09, 2019",272.31,271.65,275.00,269.61,5.90M,-0.33%
4,"Apr 08, 2019",273.20,277.69,281.16,270.44,10.41M,-0.64%
5,"Apr 05, 2019",274.96,269.86,276.10,266.11,13.04M,2.68%
6,"Apr 04, 2019",267.78,261.89,271.20,260.59,23.72M,-8.23%
7,"Apr 03, 2019",291.81,287.32,296.17,287.17,7.93M,2.07%
8,"Apr 02, 2019",285.88,288.30,289.44,283.88,5.48M,-1.14%
9,"Apr 01, 2019",289.18,282.62,289.20,281.28,8.11M,3.33%


In [25]:
from sdata import *

ModuleNotFoundError: No module named 'telepot'

In [ ]:
mylist()

In [ ]:
investing_get_historical_data('EUR/USD', '01/01/2015')